In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from data import get_state_ndays_data
import pandas
import random

In [2]:
def loss(pred,true):
    y_true, y_pred = np.array(true), np.array(pred)
    return (np.mean(np.abs((y_true - y_pred) / y_true)))*100

class SIRModel:
    def __init__(self):
        self.pred_days=30
        self.california_like_states=["California",
        "Michigan",
        "New Jersey",
        "New York",
        "New Hampshire"]
        
        self.hawaii_like_states=["Hawaii",
            "Connecticut",
            "Michigan",
            "New York",
            "Kansas",
            "New Jersey",
            "Pennsylvania"]

    def fit(self, confirm, death, N,state):
        self.N = N
        E0=3
        UI0=0.25
        # returned solution is [S, E,UI,RI,R]
        self.__hyperParametersInit = [self.N, 
                                      E0 * confirm[0], 
                                      confirm[0]*UI0, 
                                      confirm[0]-death[0],
                                      death[0]]
        # beta, gamma, sigma, mu, E0, UI0, phi,a
        if state in self.california_like_states:
            self.__parametersInit = [1e-3, 8e-2, 8e-1, 3e-1, 3,0.25,8e-2,0.2]
            self.__bounds = [(1e-5, 2), (1e-5, 1), (1e-5, 1e-1), (1e-5, 1e-2),(1, 5),(1e-2,1),(1e-7, 1e-1),(1e-5,1)]
        elif state in self.hawaii_like_states:
            self.__parametersInit = [1e-3, 8e-2, 8e-1, 3e-1, 3,0.25,8e-2,0.2]
            self.__bounds = [(1e-5, 2), (1e-5, 1), (1e-5, 1e-1), (1e-5, 1e-1),(1, 5),(1e-2,1),(1e-7, 1e-2),(1e-5,1)]
        else:
            self.__parametersInit = [1e-3, 8e-2, 8e-1, 3e-1, 3,0.25,8e-2,0.2]
            self.__bounds = [(1e-5, 2), (1e-5, 1), (1e-5, 1), (1e-5, 1),(1, 5),(1e-2,1),(1e-7, 1),(1e-5,1)]
        assert len(confirm) == len(death)
        self.__trainingData = [confirm[:-self.pred_days], death[:-self.pred_days]]
        self.__testingData = [confirm[-self.pred_days:], death[-self.pred_days:]]
        self.__loss_contribution = np.array([[1], [1]])  # whether I/D is more important in MSE
        optimal_best_val=9999999
        optimal_best=None
        for i in range(15):
            optimal = minimize(self.__iteration, self.__parametersInit, method='L-BFGS-B', bounds=self.__bounds,tol=1e-8)
            print(optimal.fun)
            print(optimal.x)
            if optimal.fun<0.3:
                optimal_best=optimal
                break
            if optimal.fun<optimal_best_val:
                optimal_best=optimal
        optimal=optimal_best
        self.__parameters = optimal.x
        self.__iteration(self.__parameters)
        tmp_train_result = self.gather_results_train(sz=len(confirm))
        print(optimal.fun)
        print(optimal.x)
        
        

    def __make_cal_gradients(self, parameters):
        beta, gamma,sigma, mu,_,_,phi,  _ = parameters
        def calc_grad(_, y):
            S, E, UI, RI, _ = y
            #TODO: modified
            return [-beta*S*((UI+E+RI))/self.N, #dS
                    beta*S*((UI+E+RI))/self.N-sigma*E, #dE
                    sigma*E-phi*UI-mu*UI, #dUI
                    mu*UI-gamma*RI, #dRI
                    gamma*RI] #dR

        return calc_grad

    def __iteration(self, parameters):
        try:
            calc_grad = self.__make_cal_gradients(parameters)
            [confirm, death] = self.__trainingData
            sz=len(confirm)
            self.__hyperParametersInit = [self.N, 
                              parameters[-4] * confirm[0], 
                              confirm[0]*parameters[-3], 
                              confirm[0]-death[0]/parameters[-1],
                              death[0]/parameters[-1]]
    
            if (confirm[0]-death[0]/parameters[-1])<0: return 9999+random.randint(1,100)
            sol = solve_ivp(calc_grad, [0, sz], self.__hyperParametersInit, t_eval=np.arange(0, sz, 1))
            pred_results=sol.y
            pred_death=[parameters[-1]*recover for recover in pred_results[-1]]
            pred_confirm=pred_results[-2]+pred_results[-1]
            for x in pred_results:
                for t in x:
                    if t<0:
                        return 9999+random.randint(1,100)
            res=(loss(pred_confirm,confirm)+(loss(pred_death,death)))/2
#             if res<2: print(res)
            return res
        except:
            return 9999+random.randint(1,100)
        
    def gather_results_train(self,sz=45):
        calc_grad = self.__make_cal_gradients(self.__parameters)
        solution = solve_ivp(calc_grad,[0, sz],self.__hyperParametersInit, t_eval=np.arange(0, sz, 1))
        return solution.y
    
    def gather_results_test(self,training_results,sz=30):
        sz=sz+1 #extra day for the first day given
        calc_grad = self.__make_cal_gradients(self.__parameters)
        init_for_test=[item[-1] for item in training_results]
        solution = solve_ivp(calc_grad,[0, sz],init_for_test, t_eval=np.arange(0, sz, 1))
        return [item[1:] for item in solution.y]
   
    def to_submission(self,total_days,predict_days):
        
        train_result = self.gather_results_train(sz=(total_days-predict_days))
        train_pred_confirm=(train_result[-1]+train_result[-2]).tolist()

        train_pred_death=[self.__parameters[-1]*recover for recover in train_result[-1]]

        test_result = model.gather_results_test(train_result,sz=predict_days)
        test_pred_confirm=(test_result[-1]+test_result[-2]).tolist()
        
        test_pred_death=[self.__parameters[-1]*recover for recover in test_result[-1]]
        
        return [[train_pred_confirm[-7:],train_pred_death[-7:]],
                [test_pred_confirm[:26],test_pred_death[:26]]]
        
        
        

if __name__ == '__main__':
    
    df_test_given=pandas.read_csv("test.csv")
#     df_population=pandas.read_csv("state_population_2019.csv",header=None)
    populations=[4903185, 731545, 7278717, 3017804, 
                 39512223, 5758736, 3565287, 973764, 
                 21477737, 10617423, 1415872, 1787065, 
                 12671821, 6732219, 3155070, 2913314, 
                 4467673, 4648794, 1344212, 6045680, 
                 6892503, 9986857, 5639632, 2976149, 
                 6137428, 1068778, 1934408, 3080156, 
                 1359711, 8882190, 2096829, 19453561, 
                 10488084, 762062, 11689100, 3956971, 
                 4217737, 12801989, 1059361, 5148714, 
                 884659, 6829174, 28995881, 3205958, 
                 623989, 8535519, 7614893, 1792147, 
                 5822434, 578759]
    
    state_list=df_test_given["Province_State"][:50].tolist()
    
#     not_okay=["Connecticut",
#             "California",
#             "Hawaii",
#             "Michigan",
#             "New York",
#             "Kansas",
#             "New Jersey",
#             "Massachusetts",
#             "Pennsylvania"]
    total_pred_confirm=[-1]*1300
    total_pred_death=[-1]*1300

    total_valid_confirm=[-1]*350
    total_valid_death=[-1]*350
    for i,State in enumerate(state_list):
        n=populations[i]
        if State=="District of Columbia":
            continue
#         if State in not_okay:
#             continue

        confirm_sep=[0]*30
        death_sep=[0]*30

        print(State)
        model = SIRModel()
        ## data from california, Parameter not tuned
        df = pandas.read_csv("train.csv")

        state_df=get_state_ndays_data(df, State, "05-13-2020", "08-31-2020")
        confirm=state_df['Confirmed'].to_numpy()
        death=state_df['Deaths'].to_numpy()

        total_days=60
        predict_days=30

        confirm_total=np.append(confirm[-(total_days-predict_days):],confirm_sep)
        death_total=np.append(death[-(total_days-predict_days):],death_sep)
        model.fit(confirm_total, death_total, n,State)

        valid_res,pred_res=model.to_submission(total_days,predict_days)
        sub_confirm,sub_death=pred_res
        v_confirm,v_death=valid_res
        
        for k in range(len(sub_confirm)):
            total_pred_confirm[k*50+i]=int(sub_confirm[k])
            total_pred_death[k*50+i]=int(sub_death[k])
        for k in range(len(v_confirm)):
            total_valid_confirm[k*50+i]=int(v_confirm[k])
            total_valid_death[k*50+i]=int(v_death[k])
        print("="*5)
        print("validation:")
        print(v_confirm)
        print("prediction:")
        print(sub_confirm)
        print("="*5)
    
    df_submission=df_test_given.iloc[:,:2]
    df_submission["Confirmed"]=total_pred_confirm
    df_submission["Deaths"]=total_pred_death
    
    df_validation=df_test_given.iloc[:350,:2]
    df_validation["Confirmed"]=total_valid_confirm
    df_validation["Deaths"]=total_valid_death
    
    
    df_new=df_submission.drop(columns="Province_State")
    df_new.to_csv("submission.csv",index=False)
    
    df_new=df_validation.drop(columns="Province_State")
    df_new.to_csv("validation.csv",index=False)
        
    

Alabama
0.9889899850034982
[0.08335648 0.00762514 0.17828955 0.05786338 1.47253711 0.06527612
 0.76886605 0.04127329]
0.9847124865289923
[0.08214166 0.00623125 0.18011229 0.05703723 1.47687583 0.06579313
 0.76736817 0.04654003]
0.9725513168991882
[0.07594698 0.00371941 0.1788783  0.06011575 1.45288964 0.06425737
 0.78449962 0.06615836]
2.7970089372715803
[1.30196825e-04 4.53077573e-03 2.71923002e-01 8.43250303e-02
 1.68178421e+00 9.17958782e-02 6.75916835e-01 5.76214696e-02]
2.796870631686435
[1.37377556e-04 4.53206175e-03 2.71913592e-01 8.43216859e-02
 1.68176271e+00 9.17932213e-02 6.75925847e-01 5.76167197e-02]
0.9777793183974308
[0.07748643 0.00404467 0.18268133 0.05720846 1.48399239 0.06621733
 0.76624645 0.0623314 ]
0.9870219637410744
[0.08316912 0.00690566 0.17940533 0.05710508 1.47519701 0.06572492
 0.76769377 0.04371198]
0.9848779278011669
[0.08274033 0.00654405 0.18053921 0.05737506 1.4720872  0.06568147
 0.77100901 0.04515226]
0.9729686289225974
[0.07622523 0.00367894 0.17699

,ForecastID,Confirmed,Deaths
0,0,123923,2217
1,1,5421,35
2,2,202329,5079
3,3,60531,799
4,4,749566,13623
...,...,...,...
1295,1295,140633,2995
1296,1296,85988,2381
1297,1297,13105,310
1298,1298,94171,1369
